# Дектораторы

В этом домашнем задании мы напишем собственные дектораторы, которые будут менять системные объекты. Но для начала мы с вами познакомимся с функцией `write`.

In [1]:
import sys
from datetime import datetime
import io

Это метод объектов file-like классов, то есть классов, которые реализуют семантику "Меня можно создать, из меня можно прочитать и в меня можно записать".

Самый главный пример такого объекта -- объект `file`, являющийся результатом вызова фукнции `open()`. Для простоты и универсальности взаимодействия, стандартный ввод и стандартный вывод тоже являются файлами, из которых можно читать и в которые можно писать. 

Как вы могли заметить, функция возвращает число записанных байт. Это важная часть контракта, которую нужно поддержать, если вы хотите как-то подменять эту функцию.

## Задача 1

Для начала, давайте подменим метод `write` у объекта `sys.stdin` на такую функцию, которая перед каждым вызовом оригинальной функции записи данных в `stdout` допечатывает к тексту текущую метку времени.

In [2]:
original_write = sys.stdout.write
def my_write(string_text):
    output = io.StringIO()
    if string_text!='\n':
        output.write("[{}]:".format(datetime.now()))
    output.write(string_text)
    original_write(output.getvalue())
    output.close()
sys.stdout.write = my_write

In [3]:
print('11')
print('12')

[2022-11-23 13:55:42.079824]:11
[2022-11-23 13:55:42.079937]:12


In [4]:
sys.stdout.write = original_write
print('11')
print('12')

11
12


## Задача 2

Упакуйте только что написанный код в декторатор. Весь вывод фукнции должен быть помечен временными метками так, как видно выше.

In [5]:
def timed_output(function):
    def wrapper(string_text):
        output = io.StringIO()
        if string_text!='\n':
            output.write("[{}]:".format(datetime.now()))
        output.write(' ')
        sys.stdout.write(output.getvalue())
        output.close()
        function(string_text)
    return wrapper     

In [6]:
@timed_output
def print_greeting(name):
    print(f'Hello, {name}!')

In [7]:
print_greeting("Nikita")

[2022-11-23 13:55:51.880736]: Hello, Nikita!


In [8]:
name = "Nikita"
print(f"Hello, {name}!")

Hello, Nikita!


## Задача 3

Напишите декторатор, который будет перенаправлять вывод фукнции в файл. 

Подсказка: вы можете заменить объект sys.stdout каким-нибудь другим объектом.

In [9]:
def redirect_output(filepath):
    def decorate(function):
        def wrapper():
            original = sys.stdout
            sys.stdout = open(filepath, 'w')
            function()
            sys.stdout = original
        return wrapper
    return decorate

In [10]:
@redirect_output('./function_output.txt')
def calculate():
    for power in range(1, 5):
        for num in range(1, 20):
            print(num ** power, end=' ')
        print()

In [11]:
calculate()

In [12]:
%cat function_output.txt

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
1 4 9 16 25 36 49 64 81 100 121 144 169 196 225 256 289 324 361 
1 8 27 64 125 216 343 512 729 1000 1331 1728 2197 2744 3375 4096 4913 5832 6859 
1 16 81 256 625 1296 2401 4096 6561 10000 14641 20736 28561 38416 50625 65536 83521 104976 130321 
